In [5]:
import pandas as pd
import random


#### Marilyn: 
- Imported the pandas and random librabries
- Simulated the transactions
- Saved the transactions to a csv file called  "'supermarket_transactions.csv'"

In [6]:
#define the pool of items that can appear in a transaction
item_pool = ['Milk', 'Eggs', 'Butter', 'Cheese', 'Bananas', 'Bread', 'Apples', 'Chicken','pork', 'beef', 'strawberries','pineapples', 'watermelon','lemon','melon', 'avocadoes', 'dragonfruit', 'orange', 'kiwi', 'flour', 'rice', 'sugar','salt','noodles','potato','onion','fruitjuice','garlic', 'okra','cucumbers']


In [7]:
# for reproducibility
random.seed(42)
 # List to hold transactions
transactions = []
# Generate 3000 fake transactions, each with 2–7 random items and store them in a list
for _ in range(3000): 
    num_items = random.randint(2, 7) 
    transaction = random.sample(item_pool, num_items)
    transactions.append(transaction) 

# Convert the transactions into a DataFrame where each row = one transaction and items separated by comma)
transactions_df = pd.DataFrame({'Transaction': [', '.join(items) for items in transactions]})

# Save to CSV
transactions_df.to_csv('supermarket_transactions.csv', index=False)
# Display the first few transactions

transactions_df.head()

,Transaction
0,"Cheese, Milk, noodles, pork, Chicken, onion, B..."
1,"Cheese, sugar, noodles, orange, Butter, kiwi, ..."
2,"Milk, Butter"
3,"Chicken, dragonfruit, flour"
4,"orange, Apples"
